In [1]:
import pandas as pd
import requests
import json
from pyproj import Transformer
from math import radians, cos, sin, asin, sqrt
import os.path
import time

datapath='data/'
secret_file = 'secrets/gcloud.json'

origin="Haifa Hof Hacarmel"
origin_coords = 2011274544.0
MODE='driving'
MODE='transit'
MAX_ELEM = 25

filename = f'{datapath}dist_{MODE}_{origin}.csv'
assert not os.path.isfile(filename), "Data already exists, will not re-fetch it to reduce costs"

with open(secret_file) as json_file:
    secrets = json.load(json_file)
    api_key=secrets['api_key']

In [2]:
transformer = Transformer.from_crs('epsg:2039', 'epsg:4326')

def get_distances(origin, destinations, api_key):
    response = requests.get(
                    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?',
                    params = {
                        'origins' : origin,
                        'destinations' : ('|').join(destinations),
                        'units' : 'metric',
                        'mode' : MODE,
                        'key' :  api_key
                            })
    return response

def coords_to_lonlat(c):
    s = f'{c:.0f}'

    if s == 'nan':
        return None, None, None
    assert len(s) == 10, "Input string must be 10 characters long"
    X = int(s[:5] + '0')
    Y = int(s[5:] + '0')

    lat, lon = transformer.transform(X, Y)
    return lon, lat, f'{lat:.4f},{lon:.4f}'

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [3]:
# source: https://www.cbs.gov.il/he/publications/Pages/2019/יישובים-בישראל.aspx
places = pd.read_excel(datapath + 'places-cbs.xlsx')

places.rename(columns={'שם יישוב באנגלית' : 'name', 'סך הכל אוכלוסייה 2021' : 'pop', 'קואורדינטות' : 'coords'}, inplace=True)

places[['lon', 'lat', 'lonlat']] = places['coords'].apply(lambda x: pd.Series(coords_to_lonlat(x)))

dests = places.query('pop > 100 and pop < 1000').copy().reset_index().drop(columns='index')
print(f'{len(dests)} destinations')
orig_lon, orig_lat, orig_lonlat = coords_to_lonlat(origin_coords)

698 destinations


In [4]:
dests['distance'] = None
dests['duration'] = None
dests['g_orig'] = None
dests['g_dest'] = None
dests['crow_dist'] = None
dests['mode'] = MODE

s_idx = 0
dlist = dests['lonlat'].to_list()
while s_idx < len(dlist):
    print(s_idx, end='...')
    e_idx = min(s_idx+MAX_ELEM, len(dests))
    r = range(s_idx, e_idx)
    dests_call = dlist[s_idx:e_idx]
    response = get_distances(origin, dests_call, api_key)
    time.sleep(0.5)
    matrix = response.json()
    DESTINATIONS=matrix['destination_addresses']
    ORIGIN=matrix['origin_addresses'][0]
    matrix = matrix['rows'][0]['elements']

    o_origin, o_dest, o_distance, o_duration, o_crow = [], [], [], [], []

    for i, element in enumerate(matrix):
        o_origin.append(ORIGIN)
        o_dest.append(DESTINATIONS[i])
        o_crow.append(1000.*haversine(orig_lon, orig_lat, dests.loc[s_idx+i, 'lon'], dests.loc[s_idx+i, 'lat']))
        if element['status'] == 'OK':
            o_distance.append(element['distance']['value'])
            o_duration.append(element['duration']['value'])
        else:
            o_distance.append(None)
            o_duration.append(None)

    dests.loc[r, 'distance'] = o_distance
    dests.loc[r, 'duration'] = o_duration
    dests.loc[r, 'g_orig'] = o_origin
    dests.loc[r, 'g_dest'] = o_dest
    dests.loc[r, 'crow_dist'] = o_crow

    
    s_idx += MAX_ELEM


0...25...50...75...100...125...150...175...200...225...250...275...300...325...350...375...400...425...450...475...500...525...550...575...600...625...650...675...

In [8]:
dests['speed'] = dests['crow_dist']/dests['duration']*3.6
dests['ratio'] = dests['crow_dist']/dests['distance']
dests.sort_values(by='ratio')[['distance', 'duration', 'speed', 'crow_dist', 'name']]

,distance,duration,speed,crow_dist,name
322,40710,5188,6.575521,9476.056328,Kefar Hasidim Bet
392,132315,9542,12.005874,31822.23587,Mikhmannim
321,40305,4896,7.219109,9817.98867,Kefar Hasidim Alef
301,29599,4292,6.314556,7528.35426,Kefar Bialik
73,31294,5503,5.285647,8079.699177,Bet Oren
...,...,...,...,...,...
518,None,None,NaN,223750.799847,Iddan
533,None,None,NaN,239540.556913,En Yahav
543,None,None,NaN,52812.087875,Amuqqa
594,None,None,NaN,64791.092863,Qiryat Shelomo


In [13]:
dests.query('duration > 1800').sort_values(by='speed')[['distance', 'duration', 'speed', 'crow_dist', 'name']][0:20]

,distance,duration,speed,crow_dist,name
73,31294,5503,5.285647,8079.699177,Bet Oren
311,37543,6564,5.557127,10132.494824,Kefar HaMakkabbi
25,36932,6115,5.847773,9933.092292,Usha
301,29599,4292,6.314556,7528.35426,Kefar Bialik
322,40710,5188,6.575521,9476.056328,Kefar Hasidim Bet
312,41536,5845,6.614194,10738.879001,Kefar HaNo'ar Hadati
215,47224,6505,6.927169,12517.009702,Khawaled
321,40305,4896,7.219109,9817.98867,Kefar Hasidim Alef
597,49404,6940,7.303869,14080.236313,Ras Ali
56,35897,4785,8.879604,11802.474054,Afeq


In [12]:
dests.to_csv(filename, index=False)